ATIVIDADE 1

('id':1,'nome':'Bob', 'idade':45,'gen':'M')
('id':2,'nome':'Gloria', 'idade':43,'gen':'F')
('id':4,'nome':'Albert', 'idade':28,'gen':'M')
('id':5,'nome':'Laura', 'idade':33,'gen':'F')
('id':8,'nome':'Simone', 'idade':18,'gen':'T')
('id':12,'nome':'Marta', 'idade':45,'gen':'F')
('id':45,'nome':'Jairo', 'idade':82,'gen':'M')
('id':13,'nome':'Teste', 'idade':38,'gen':'T')

Criar um DataFrame a partir das informações acima. Em seguida, fazer a análise utilizando o PySpark.


1) Verifique as pessoas com idade superior a 30 anos


2) Verificar apenas as pessoas do sexo F


3) Verifique a quantidade de pessoas por sexo


4) Calcular a média de idade das pessoas


5) Fazer uma seleção apenas do nome e idade das pessoas ordenando por nome


6) Mostrar um ranking de pessoas por idade (Do mais novo para o mais velho)

In [ ]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 34 kB/s 
     |████████████████████████████████| 198 kB 62.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=61da5d9829598fa69e7c8017f5da6fae3c62ab9d7d597da1ceacddf150451231
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, FloatType

In [ ]:
spark = SparkSession.builder.master('local').appName('learning-df').config('spark.ui.port', '4050').getOrCreate()

In [ ]:
dados = [
         (1, 'Bob', 45, 'M'), 
         (2,'Gloria', 43, 'F'), 
         (4,'Albert', 28, 'M'), 
         (5,'Laura', 33, 'F'), 
         (8,'Simone', 18, 'T'), 
         (12,'Marta', 45, 'F'), 
         (45,'Jairo', 82, 'M'), 
         (13,'Teste', 38, 'T')
]

schema= (
    StructType([
                StructField('id', IntegerType(), True),
                StructField('nome', StringType(), True),
                StructField('idade', IntegerType(), True),
                StructField('genero', StringType(), True)
    ])
)

In [ ]:
df = spark.createDataFrame(data=dados, schema=schema)

In [ ]:
df.show()
df.printSchema()

+---+------+-----+------+
| id|  nome|idade|genero|
+---+------+-----+------+
|  1|   Bob|   45|     M|
|  2|Gloria|   43|     F|
|  4|Albert|   28|     M|
|  5| Laura|   33|     F|
|  8|Simone|   18|     T|
| 12| Marta|   45|     F|
| 45| Jairo|   82|     M|
| 13| Teste|   38|     T|
+---+------+-----+------+

root
 |-- id: integer (nullable = true)
 |-- nome: string (nullable = true)
 |-- idade: integer (nullable = true)
 |-- genero: string (nullable = true)



In [ ]:
#1) Verifique as pessoas com idade superior a 30 anos
df.select(F.col('nome'), F.col('idade')).filter(F.col('idade') > 30).show()

+------+-----+
|  nome|idade|
+------+-----+
|   Bob|   45|
|Gloria|   43|
| Laura|   33|
| Marta|   45|
| Jairo|   82|
| Teste|   38|
+------+-----+



In [ ]:
#2) Verificar apenas as pessoas do sexo F
df.select(F.col('nome'), F.col('genero')).filter(F.col('genero') ==  'F').show()

+------+------+
|  nome|genero|
+------+------+
|Gloria|     F|
| Laura|     F|
| Marta|     F|
+------+------+



In [ ]:
#3) Verifique a quantidade de pessoas por sexo
df.groupBy(F.col('genero')).count().show()

+------+-----+
|genero|count|
+------+-----+
|     F|    3|
|     T|    2|
|     M|    3|
+------+-----+



In [ ]:
#4) Calcular a média de idade das pessoas
media = df.select(F.mean('idade')).collect()[0][0]
print(f'Média das idades: {media:.2f}')

Média das idades: 41.50


In [87]:
df.groupBy().mean('idade').show()

+----------+
|avg(idade)|
+----------+
|      41.5|
+----------+



In [89]:
df.select(F.mean(F.col('idade')).alias('media_idades')).show()

+------------+
|media_idades|
+------------+
|        41.5|
+------------+



In [ ]:
#5) Fazer uma seleção apenas do nome e idade das pessoas ordenando por nome
df.select(F.col('nome'), F.col('idade')).orderBy('nome').show()

+------+-----+
|  nome|idade|
+------+-----+
|Albert|   28|
|   Bob|   45|
|Gloria|   43|
| Jairo|   82|
| Laura|   33|
| Marta|   45|
|Simone|   18|
| Teste|   38|
+------+-----+



In [ ]:
df.select(F.col('nome'), F.col('idade')).orderBy(F.col('nome').desc()).show()


+------+-----+
|  nome|idade|
+------+-----+
| Teste|   38|
|Simone|   18|
| Marta|   45|
| Laura|   33|
| Jairo|   82|
|Gloria|   43|
|   Bob|   45|
|Albert|   28|
+------+-----+



In [ ]:
#6) Mostrar um ranking de pessoas por idade (Do mais novo para o mais velho)
df.select(F.col('nome'), F.col('idade')).orderBy(F.col('idade').asc()).show()

+------+-----+
|  nome|idade|
+------+-----+
|Simone|   18|
|Albert|   28|
| Laura|   33|
| Teste|   38|
|Gloria|   43|
|   Bob|   45|
| Marta|   45|
| Jairo|   82|
+------+-----+



In [ ]:
from pyspark.sql.window import Window

In [ ]:
w0 = Window.partitionBy(F.col('genero')).orderBy('idade')
df.withColumn('row_number', F.row_number().over(w0)).show()

+---+------+-----+------+----------+
| id|  nome|idade|genero|row_number|
+---+------+-----+------+----------+
|  5| Laura|   33|     F|         1|
|  2|Gloria|   43|     F|         2|
| 12| Marta|   45|     F|         3|
|  4|Albert|   28|     M|         1|
|  1|   Bob|   45|     M|         2|
| 45| Jairo|   82|     M|         3|
|  8|Simone|   18|     T|         1|
| 13| Teste|   38|     T|         2|
+---+------+-----+------+----------+



In [ ]:
df.withColumn('rank',  F.dense_rank().over(w0)).show()

+---+------+-----+------+----+
| id|  nome|idade|genero|rank|
+---+------+-----+------+----+
|  5| Laura|   33|     F|   1|
|  2|Gloria|   43|     F|   2|
| 12| Marta|   45|     F|   3|
|  4|Albert|   28|     M|   1|
|  1|   Bob|   45|     M|   2|
| 45| Jairo|   82|     M|   3|
|  8|Simone|   18|     T|   1|
| 13| Teste|   38|     T|   2|
+---+------+-----+------+----+



In [90]:
w1 = Window.orderBy('idade')
df.withColumn('rank', F.rank().over(w1)).show()

+---+------+-----+------+----+
| id|  nome|idade|genero|rank|
+---+------+-----+------+----+
|  8|Simone|   18|     T|   1|
|  4|Albert|   28|     M|   2|
|  5| Laura|   33|     F|   3|
| 13| Teste|   38|     T|   4|
|  2|Gloria|   43|     F|   5|
|  1|   Bob|   45|     M|   6|
| 12| Marta|   45|     F|   6|
| 45| Jairo|   82|     M|   8|
+---+------+-----+------+----+



In [94]:
idadecresc  = Window.orderBy("idade")
df.withColumn('rank',F.rank().over(idadecresc)).show()

+---+------+-----+------+----+
| id|  nome|idade|genero|rank|
+---+------+-----+------+----+
|  8|Simone|   18|     T|   1|
|  4|Albert|   28|     M|   2|
|  5| Laura|   33|     F|   3|
| 13| Teste|   38|     T|   4|
|  2|Gloria|   43|     F|   5|
|  1|   Bob|   45|     M|   6|
| 12| Marta|   45|     F|   6|
| 45| Jairo|   82|     M|   8|
+---+------+-----+------+----+



In [95]:
idadecresc2  = Window.orderBy("idade")
df.withColumn('rank',F.dense_rank().over(idadecresc2)).show()

+---+------+-----+------+----+
| id|  nome|idade|genero|rank|
+---+------+-----+------+----+
|  8|Simone|   18|     T|   1|
|  4|Albert|   28|     M|   2|
|  5| Laura|   33|     F|   3|
| 13| Teste|   38|     T|   4|
|  2|Gloria|   43|     F|   5|
|  1|   Bob|   45|     M|   6|
| 12| Marta|   45|     F|   6|
| 45| Jairo|   82|     M|   7|
+---+------+-----+------+----+

